# Variability of the inorganic carbon system in a large coastal lagoon of the southern Gulf of Mexico

### José Andrés Martínez-Trejo<sup>1</sup>, Jose-Gilberto Cardoso-Mohedano<sup>2,*</sup>, Joan Albert Sanchez-Cabeza<sup>3</sup>, José Martin Hernández Ayón<sup>4</sup>, Ana Carolina Ruiz-Fernández<sup> 3 </sup>, Mario Alejandro Gómez-Ponce <sup>2</sup> , Linda Barranco-Servín<sup>5</sup>, Daniel Pech<sup>5</sup>

<sup> 1 </sup> Posgrado en Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México; Avenida Ciudad Universitaria 3000, C.P. 04510, Coyoacán, Ciudad de México, México

<sup> 2 </sup>  Estación el Carmen, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Carretera Carmen-Puerto Real km. 9.5, 24157 Ciudad del Carmen, Campeche, México

<sup> 3 </sup>  Unidad Académica Mazatlán, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Calzada Joel Montes Camarena s/n, Colonia Playa Sur, Mazatlán 82000, Sinaloa, México

<sup> 4 </sup>  Instituto de Investigaciones Oceanológicas, Universidad Autónoma de Baja California, Km. 103 CarreteraTijuana-Ensenada, Ensenada, Baja California, México

<sup> 5 </sup> Laboratorio de Biodiversidad Marina y Cambio Climático (BIOMARCCA), El Colegio de la Frontera Sur, Lerma, Campeche, México. 

 \*Corresponding Author: gcardoso@cmarl.unam.mx Phone:+52-938-38-31847

[doi:XXXXX](https://falta)


- Read csv raw data file

In [1]:
import pandas as pd 
url = "https://metadata.icmyl.unam.mx/bitstream/handle/20.500.12201/11367/MartinezTrejo_etal_2023_TA_DIC_RawData.csv?sequence=1&isAllowed=y"
myCO2data = pd.read_csv(url, sep=",", header=0,  decimal=".",
                        encoding = 'utf-8')

- PyCO2SYS configuration from estuarine waters (Humphreys et al. 2022)

In [20]:
import PyCO2SYS as pyco2
myCO2sysParamsDICTA_data = {
    "par1": myCO2data["DIC_micromol_kg"], # DIC measured in the lab in μmol/kg-sw
    "par2": myCO2data["TA_micromol_kg"],         # TA measured in the lab, Total scale
    "par1_type": 2,         # tell PyCO2SYS: "par2 is a DIC value" 
    "par2_type": 1,         # tell PyCO2SYS: "par1 is a TA value"
    # Fields conditions
    "salinity": myCO2data["Sal_psu"],           #  in-situ salinity in PSU
    "temperature_out": myCO2data["Temp_C"],    # in-situ temperature (output conditions) in °C
    "pressure_out": 0.0,
    # Settings
    "opt_pH_scale" : 1,     # Total pH (Wolf-Gladrow et al. 2007)
    "opt_k_carbonic": 14,    # 0 < T < 50 °C, 1 < S < 50, Seawater scale, real seawater (Millero F.J. 2010)    
    "temperature": 25      # lab temperature (input conditions) in °C
}

- Run PyCO2SYS

In [21]:
CO2results= pyco2.sys(**myCO2sysParamsDICTA_data)

# References
- Humphreys, M. P., Lewis, E. R., Sharp, J. D., & Pierrot, D. (2022). PyCO2SYS v1.8: Marine carbonate system calculations in Python. Geoscientific Model Development, 15(1), 15–43. https://doi.org/10.5194/gmd-15-15-2022
- Millero, F. J. (2010). Carbonate constants for estuarine waters. Marine and Freshwater Research, 61(2), 139. https://doi.org/10.1071/MF09254
- Wolf-Gladrow, D. A., Zeebe, R. E., Klaas, C., Körtzinger, A., & Dickson, A. G. (2007). Total alkalinity: The explicit conservative expression and its application to biogeochemical processes. Marine Chemistry, 106(1–2), 287–300. https://doi.org/10.1016/j.marchem.2007.01.006

In [22]:
type(CO2results)

dict

In [28]:
for clave in CO2results:
    print(clave)

par1
par1_type
par2
par2_type
alkalinity
dic
opt_k_bisulfate
opt_k_carbonic
opt_k_fluoride
opt_total_borate
opt_gas_constant
opt_pH_scale
opt_buffers_mode
salinity
temperature
pressure
total_ammonia
total_borate
total_calcium
total_fluoride
total_phosphate
total_silicate
total_sulfate
total_sulfide
peng_correction
gas_constant
total_alpha
total_beta
pressure_atmosphere
pressure_atmosphere_out
pH
pCO2
fCO2
bicarbonate
carbonate
aqueous_CO2
xCO2
alkalinity_borate
hydroxide
alkalinity_phosphate
alkalinity_silicate
alkalinity_ammonia
alkalinity_sulfide
hydrogen_free
revelle_factor
saturation_calcite
saturation_aragonite
pH_total
pH_sws
pH_free
pH_nbs
gamma_dic
beta_dic
omega_dic
gamma_alk
beta_alk
omega_alk
isocapnic_quotient
isocapnic_quotient_approx
psi
substrate_inhibitor_ratio
alkalinity_alpha
alkalinity_beta
HCO3
CO3
CO2
BOH4
BOH3
OH
Hfree
H3PO4
H2PO4
HPO4
PO4
H3SiO4
H4SiO4
NH3
NH4
HS
H2S
HSO4
SO4
HF
F
alpha
alphaH
beta
betaH
k_CO2
k_carbonic_1
k_carbonic_2
k_water
k_borate
k_bisulfat

In [29]:
dates = pd.date_range(start='2023-01-01', end='2023-01-05', freq='D')
values = [10, 20, 30, 40, 50]
df = pd.DataFrame({'Date': dates, 'Value': values})

# create a sample dictionary
d = {'2023-01-01': 'Holiday', '2023-01-03': 'Vacation'}

# convert the dictionary to a dataframe
df2 = pd.DataFrame(list(d.items()), columns=['Date', 'Event'])

In [39]:
myInteresVarsDictonary = {"pH": CO2results["pH"],
       "aragonite": CO2results["saturation_aragonite"],
       "pCO2":CO2results["pCO2_out"]}
myInteresVarPandas = pd.DataFrame.from_dict(myInteresVarsDictonary, orient='columns')
myInteresVarPandas.index = myCO2data.index

In [40]:
myInteresVarPandas

,pH,aragonite,pCO2
0,7.686758,0.801063,3401.042881
1,7.490880,1.256185,3634.365201
2,7.614324,1.168559,3194.128854
3,8.181676,5.225870,754.676675
4,7.640009,1.816427,1834.384666
...,...,...,...
101,7.809464,2.215801,1094.844098
102,7.951301,2.923447,783.190736
103,8.017392,3.469024,625.390704
104,7.954982,2.910010,762.204837
